In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-1.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-1        4    2    4    0    2    5     1   0   2  ...  1.000000   
1      prop-1        4   12    4    1   19   44    66   2  18  ...  0.000000   
2      prop-1        4   10    4    0   13   48    45   4  13  ...  1.000000   
3      prop-1        4    2    3    0    5    7     1   0   5  ...  1.000000   
4      prop-1        4    8    1    0    4   38     0   3   1  ...  0.500000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
18466  prop-1      185   17    1    0   24   67    96   9  15  ...  0.150000   
18467  prop-1      185    9    1    0   25   49    14   7  18  ...  0.333333   
18468  prop-1      185    6    3    0   10   29     0   0  10  ...  0.800000   
18469  prop-1      185    5    5    0   28    9    10  26   2  ...  1.000000   
18470  prop-1      185   11    1    0   54   34     9  51   3  ...  0.666667   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0        0  0.941176  0.750000   1    1  10.500000       3  1.5000    0  
1        0  0.905172  0.350000   2    3  20.583333       8  2.0833    0  
2        0  0.918919  0.500000   2    3  19.100000       4  1.2000    0  
3        0  0.933333  0.750000   1    1  11.500000       4  2.0000    1  
4        0  0.000000  0.285714   0    0  30.625000       2  0.8750    0  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
18466    0  0.000000  0.250000   0    0  41.529412       6  2.6471    1  
18467    0  0.000000  0.319444   0    0  29.888889       3  1.3333    0  
18468    0  0.818182  0.400000   0    0  45.500000       6  2.1667    0  
18469    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
18470    1  0.000000  0.400000   0    0  25.545455       6  1.9091    0  

[18471 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 185]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom   ca  ce  ...       dam  \
15646  prop-1      185    3    1    0    4   11     3    1   3  ...  0.000000   
15647  prop-1      185   13    1   22   62   96    74   22  40  ...  0.750000   
15648  prop-1      185    2    1    0    2    5     1    1   1  ...  0.000000   
15649  prop-1      185    5    1    0  148    5    10  144   4  ...  0.000000   
15650  prop-1      185   18    2    0   53  146   117    1  53  ...  0.625000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ...  ..  ...       ...   
18466  prop-1      185   17    1    0   24   67    96    9  15  ...  0.150000   
18467  prop-1      185    9    1    0   25   49    14    7  18  ...  0.333333   
18468  prop-1      185    6    3    0   10   29     0    0  10  ...  0.800000   
18469  prop-1      185    5    5    0   28    9    10   26   2  ...  1.000000   
18470  prop-1      185   11    1    0   54   34     9   51   3  ...  0.666667   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
15646    0  0.000000  0.333333   0    0  22.333333       3  1.6667    0  
15647    1  0.000000  0.333333   0    0  38.230769      13  3.3077    0  
15648    0  0.000000  0.750000   0    0  17.000000       6  3.0000    0  
15649    0  0.000000  1.000000   0    0   0.000000       1  1.0000    0  
15650    0  0.448276  0.264706   1    8  64.222222      35  5.1667    1  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
18466    0  0.000000  0.250000   0    0  41.529412       6  2.6471    1  
18467    0  0.000000  0.319444   0    0  29.888889       3  1.3333    0  
18468    0  0.818182  0.400000   0    0  45.500000       6  2.1667    0  
18469    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
18470    1  0.000000  0.400000   0    0  25.545455       6  1.9091    0  

[2825 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  \
15827    6    1    0    1   16    15   1   0    5  2.000000  129  0.000000   
16556    8    1    0    5   14    10   2   3    8  0.714286   69  1.000000   
18036    2    3    0    8   22     1   0   8    2  1.000000   97  0.000000   
16914    6    3    0   11   20     9   0  11    6  0.933333  115  0.333333   
15767    0    1    0    0    0     0   0   0    0  2.000000    3  0.000000   
...    ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...   
16544    2    4    0   10    7     1   0  10    1  2.000000   62  0.000000   
17924   39    1    0   25   39   741  22   3   39  2.000000   39  0.000000   
18305   12    1    2    9   21    52   4   5   12  0.927273   93  0.800000   
17106   18    3    0   13   50    79   2  11   14  0.823529  262  0.500000   
16575    6    1    0   37  103     0   3  36    1  0.757143  864  0.500000   

       moa       mfa       cam  ic  cbm         amc  max_cc  avg_cc  
15827    0  0.000000  0.500000   0    0   20.500000       4  1.6667  
16556    3  0.000000  0.343750   0    0    7.250000       1  0.8750  
18036    0  0.857143  0.666667   2    2   47.000000       8  4.0000  
16914    0  0.886364  0.333333   2    6   17.666667       8  2.3333  
15767    0  0.000000  0.000000   0    0    0.000000       0  0.0000  
...    ...       ...       ...  ..  ...         ...     ...     ...  
16544    0  0.909091  0.750000   1    1   30.000000       1  0.5000  
17924    0  0.000000  0.623932   0    0    0.000000       1  1.0000  
18305    0  0.000000  0.222222   0    0    6.333333       1  0.9167  
17106    0  0.812500  0.166667   3   19   13.333333       7  1.7222  
16575    2  0.000000  0.700000   0    0  140.666667       1  0.6667  

[2260 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3   loc       dam  \
17906    2    2    0   11   12     1   0  11    2  1.000000    47  0.000000   
15984    8    6    0    8   30     0   4   6    2  0.825397   187  0.555556   
17473    2    2    0    5   12     1   0   5    1  1.000000   113  0.000000   
15650   18    2    0   53  146   117   1  53    3  0.966912  1190  0.625000   
15826    2    1    0   12   17     1   0  12    2  2.000000    44  1.000000   
...    ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   ...       ...   
15853    3    3    0    7   11     3   0   7    2  1.500000    50  1.000000   
16050   54    3    9   72  217  1209  23  57   22  0.980312  1542  0.217391   
16368    0    1    0    0    0     0   0   0    0  2.000000    19  0.000000   
15891    5    5    0   17   41     8   0  17    3  1.000000   120  0.666667   
17853    3    3    0   23   53     1   0  23    2  0.875000   222  0.250000   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
17906    0  0.800000  0.666667   1    1  22.000000       1  0.5000  
15984    0  0.986900  0.428571   0    0  21.250000       1  0.7500  
17473    0  0.916667  1.000000   0    0  55.000000       2  1.0000  
15650    0  0.448276  0.264706   1    8  64.222222      35  5.1667  
15826    0  0.000000  0.666667   0    0  20.500000       2  1.0000  
...    ...       ...       ...  ..  ...        ...     ...     ...  
15853    0  0.866667  0.555556   2    2  15.333333       4  1.6667  
16050    2  0.428571  0.161426   1    5  27.129630      23  2.2407  
16368    0  0.000000  0.000000   0    0   0.000000       0  0.0000  
15891    0  0.970588  0.437500   2    2  22.400000       2  1.0000  
17853    0  0.857143  0.666667   1    1  71.666667       8  2.6667  

[565 rows x 20 columns]

In [9]:
y_train = train['bug']
y_test = test['bug']

In [10]:
y_train

15827    0
16556    0
18036    0
16914    0
15767    0
        ..
16544    0
17924    0
18305    0
17106    0
16575    0
Name: bug, Length: 2260, dtype: int64

In [11]:
y_test

17906    0
15984    0
17473    0
15650    1
15826    0
        ..
15853    0
16050    0
16368    0
15891    0
17853    1
Name: bug, Length: 565, dtype: int64

In [12]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [13]:
x_train_np

array([[  6.        ,   1.        ,   0.        , ...,  20.5       ,
          4.        ,   1.6667    ],
       [  8.        ,   1.        ,   0.        , ...,   7.25      ,
          1.        ,   0.875     ],
       [  2.        ,   3.        ,   0.        , ...,  47.        ,
          8.        ,   4.        ],
       ...,
       [ 12.        ,   1.        ,   2.        , ...,   6.33333333,
          1.        ,   0.9167    ],
       [ 18.        ,   3.        ,   0.        , ...,  13.33333333,
          7.        ,   1.7222    ],
       [  6.        ,   1.        ,   0.        , ..., 140.6666667 ,
          1.        ,   0.6667    ]])

In [14]:
x_test_np

array([[ 2.        ,  2.        ,  0.        , ..., 22.        ,
         1.        ,  0.5       ],
       [ 8.        ,  6.        ,  0.        , ..., 21.25      ,
         1.        ,  0.75      ],
       [ 2.        ,  2.        ,  0.        , ..., 55.        ,
         2.        ,  1.        ],
       ...,
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 5.        ,  5.        ,  0.        , ..., 22.4       ,
         2.        ,  1.        ],
       [ 3.        ,  3.        ,  0.        , ..., 71.66666667,
         8.        ,  2.6667    ]])

In [15]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [16]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [17]:
train_x

array([[[  6.        ,   1.        ,   0.        , ...,  20.5       ,
           4.        ,   1.6667    ]],

       [[  8.        ,   1.        ,   0.        , ...,   7.25      ,
           1.        ,   0.875     ]],

       [[  2.        ,   3.        ,   0.        , ...,  47.        ,
           8.        ,   4.        ]],

       ...,

       [[ 12.        ,   1.        ,   2.        , ...,   6.33333333,
           1.        ,   0.9167    ]],

       [[ 18.        ,   3.        ,   0.        , ...,  13.33333333,
           7.        ,   1.7222    ]],

       [[  6.        ,   1.        ,   0.        , ..., 140.6666667 ,
           1.        ,   0.6667    ]]])

In [18]:
test_y

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 4, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,

In [19]:
def define_model():
    # Designing and initializing the model.
    model = Sequential()
    model.add(LSTM(200, return_sequences=True))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(50, dropout = 0.2, return_sequences=True))
    model.add(LSTM(20, return_sequences=False))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )
    return model

In [20]:
y = list(test_y)

In [21]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(10):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/propv185/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(test_x, test_y)
    total_scores.append(score) 
    predictions = model.predict(test_x)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
23/23 [==============================] - 9s 36ms/step - loss: 0.4851 - mse: 0.4851 - mae: 0.2194 - root_mean_squared_error: 0.6965 - mean_squared_logarithmic_error: 0.0990
Epoch 2/100
23/23 [==============================] - 1s 23ms/step - loss: 0.4520 - mse: 0.4520 - mae: 0.3021 - root_mean_squared_error: 0.6723 - mean_squared_logarithmic_error: 0.0952
Epoch 3/100
23/23 [==============================] - 1s 22ms/step - loss: 0.4343 - mse: 0.4343 - mae: 0.2865 - root_mean_squared_error: 0.6590 - mean_squared_logarithmic_error: 0.0922
Epoch 4/100
23/23 [==============================] - 0s 21ms/step - loss: 0.4119 - mse: 0.4119 - mae: 0.2782 - root_mean_squared_error: 0.6418 - mean_squared_logarithmic_error: 0.0867
Epoch 5/100
23/23 [==============================] - 0s 22ms/step - loss: 0.3772 - mse: 0.3772 - mae: 0.2478 - root_mean_squared_error: 0.6142 - mean_squared_logarithmic_error: 0.0794
Epoch 6/100
23/23 [==============================] - 0s 21ms/step - lo

23/23 [==============================] - 0s 21ms/step - loss: 0.2614 - mse: 0.2614 - mae: 0.1877 - root_mean_squared_error: 0.5113 - mean_squared_logarithmic_error: 0.0645
Epoch 45/100
23/23 [==============================] - 0s 20ms/step - loss: 0.2946 - mse: 0.2946 - mae: 0.1876 - root_mean_squared_error: 0.5427 - mean_squared_logarithmic_error: 0.0674
Epoch 46/100
23/23 [==============================] - 0s 20ms/step - loss: 0.2744 - mse: 0.2744 - mae: 0.1842 - root_mean_squared_error: 0.5239 - mean_squared_logarithmic_error: 0.0636
Epoch 47/100
23/23 [==============================] - 0s 20ms/step - loss: 0.2625 - mse: 0.2625 - mae: 0.1856 - root_mean_squared_error: 0.5123 - mean_squared_logarithmic_error: 0.0623
Epoch 48/100
23/23 [==============================] - 0s 21ms/step - loss: 0.2596 - mse: 0.2596 - mae: 0.1838 - root_mean_squared_error: 0.5095 - mean_squared_logarithmic_error: 0.0618
Epoch 49/100
23/23 [==============================] - 0s 21ms/step - loss: 0.2613 - mse:

23/23 [==============================] - 0s 21ms/step - loss: 0.2447 - mse: 0.2447 - mae: 0.2038 - root_mean_squared_error: 0.4946 - mean_squared_logarithmic_error: 0.0587
Epoch 89/100
23/23 [==============================] - 0s 21ms/step - loss: 0.2254 - mse: 0.2254 - mae: 0.1882 - root_mean_squared_error: 0.4748 - mean_squared_logarithmic_error: 0.0564
Epoch 90/100
23/23 [==============================] - 0s 21ms/step - loss: 0.2337 - mse: 0.2337 - mae: 0.1901 - root_mean_squared_error: 0.4834 - mean_squared_logarithmic_error: 0.0554
Epoch 91/100
23/23 [==============================] - 0s 21ms/step - loss: 0.2492 - mse: 0.2492 - mae: 0.2000 - root_mean_squared_error: 0.4992 - mean_squared_logarithmic_error: 0.0597
Epoch 92/100
23/23 [==============================] - 0s 21ms/step - loss: 0.2412 - mse: 0.2412 - mae: 0.2004 - root_mean_squared_error: 0.4911 - mean_squared_logarithmic_error: 0.0583
Epoch 93/100
23/23 [==============================] - 0s 21ms/step - loss: 0.2340 - mse:

23/23 [==============================] - 0s 21ms/step - loss: 0.2792 - mse: 0.2792 - mae: 0.1892 - root_mean_squared_error: 0.5284 - mean_squared_logarithmic_error: 0.0643
Epoch 30/100
23/23 [==============================] - 0s 21ms/step - loss: 0.2804 - mse: 0.2804 - mae: 0.1931 - root_mean_squared_error: 0.5295 - mean_squared_logarithmic_error: 0.0662
Epoch 31/100
23/23 [==============================] - 0s 21ms/step - loss: 0.2962 - mse: 0.2962 - mae: 0.1869 - root_mean_squared_error: 0.5442 - mean_squared_logarithmic_error: 0.0702
Epoch 32/100
23/23 [==============================] - 0s 21ms/step - loss: 0.2929 - mse: 0.2929 - mae: 0.1913 - root_mean_squared_error: 0.5412 - mean_squared_logarithmic_error: 0.0670
Epoch 33/100
23/23 [==============================] - 0s 21ms/step - loss: 0.2840 - mse: 0.2840 - mae: 0.1882 - root_mean_squared_error: 0.5329 - mean_squared_logarithmic_error: 0.0670
Epoch 34/100
23/23 [==============================] - 0s 21ms/step - loss: 0.2749 - mse:

23/23 [==============================] - 1s 24ms/step - loss: 0.2284 - mse: 0.2284 - mae: 0.1658 - root_mean_squared_error: 0.4779 - mean_squared_logarithmic_error: 0.0555
Epoch 74/100
23/23 [==============================] - 1s 22ms/step - loss: 0.2244 - mse: 0.2244 - mae: 0.1702 - root_mean_squared_error: 0.4737 - mean_squared_logarithmic_error: 0.0545
Epoch 75/100
23/23 [==============================] - 0s 20ms/step - loss: 0.2263 - mse: 0.2263 - mae: 0.1735 - root_mean_squared_error: 0.4757 - mean_squared_logarithmic_error: 0.0566
Epoch 76/100
23/23 [==============================] - 1s 24ms/step - loss: 0.2145 - mse: 0.2145 - mae: 0.1744 - root_mean_squared_error: 0.4631 - mean_squared_logarithmic_error: 0.0553
Epoch 77/100
23/23 [==============================] - 0s 22ms/step - loss: 0.2432 - mse: 0.2432 - mae: 0.1741 - root_mean_squared_error: 0.4931 - mean_squared_logarithmic_error: 0.0543
Epoch 78/100
23/23 [==============================] - 0s 22ms/step - loss: 0.2272 - mse:

...

23/23 [==============================] - 0s 6ms/step - loss: 0.4996 - mse: 0.4996 - mae: 0.1792 - root_mean_squared_error: 0.7068 - mean_squared_logarithmic_error: 0.1046
Epoch 18/100
23/23 [==============================] - 0s 6ms/step - loss: 0.4996 - mse: 0.4996 - mae: 0.1792 - root_mean_squared_error: 0.7068 - mean_squared_logarithmic_error: 0.1046
Epoch 19/100
23/23 [==============================] - 0s 6ms/step - loss: 0.4996 - mse: 0.4996 - mae: 0.1792 - root_mean_squared_error: 0.7068 - mean_squared_logarithmic_error: 0.1046
Epoch 20/100
23/23 [==============================] - 0s 7ms/step - loss: 0.4996 - mse: 0.4996 - mae: 0.1792 - root_mean_squared_error: 0.7068 - mean_squared_logarithmic_error: 0.1046
Epoch 21/100
23/23 [==============================] - 0s 7ms/step - loss: 0.4996 - mse: 0.4996 - mae: 0.1792 - root_mean_squared_error: 0.7068 - mean_squared_logarithmic_error: 0.1046
Epoch 22/100
23/23 [==============================] - 0s 6ms/step - loss: 0.4996 - mse: 0.499

23/23 [==============================] - 0s 6ms/step - loss: 0.4996 - mse: 0.4996 - mae: 0.1792 - root_mean_squared_error: 0.7068 - mean_squared_logarithmic_error: 0.1046
Epoch 62/100
23/23 [==============================] - 0s 6ms/step - loss: 0.4996 - mse: 0.4996 - mae: 0.1792 - root_mean_squared_error: 0.7068 - mean_squared_logarithmic_error: 0.1046
Epoch 63/100
23/23 [==============================] - 0s 6ms/step - loss: 0.4996 - mse: 0.4996 - mae: 0.1792 - root_mean_squared_error: 0.7068 - mean_squared_logarithmic_error: 0.1046
Epoch 64/100
23/23 [==============================] - 0s 6ms/step - loss: 0.4996 - mse: 0.4996 - mae: 0.1792 - root_mean_squared_error: 0.7068 - mean_squared_logarithmic_error: 0.1046
Epoch 65/100
23/23 [==============================] - 0s 6ms/step - loss: 0.4996 - mse: 0.4996 - mae: 0.1792 - root_mean_squared_error: 0.7068 - mean_squared_logarithmic_error: 0.1046
Epoch 66/100
23/23 [==============================] - 0s 6ms/step - loss: 0.4996 - mse: 0.499

23/23 [==============================] - 0s 6ms/step - loss: 0.4367 - mse: 0.4367 - mae: 0.2829 - root_mean_squared_error: 0.6608 - mean_squared_logarithmic_error: 0.0909
Epoch 4/100
23/23 [==============================] - 0s 6ms/step - loss: 0.4167 - mse: 0.4167 - mae: 0.2797 - root_mean_squared_error: 0.6455 - mean_squared_logarithmic_error: 0.0881
Epoch 5/100
23/23 [==============================] - 0s 6ms/step - loss: 0.3861 - mse: 0.3861 - mae: 0.2644 - root_mean_squared_error: 0.6213 - mean_squared_logarithmic_error: 0.0825
Epoch 6/100
23/23 [==============================] - 0s 6ms/step - loss: 0.3581 - mse: 0.3581 - mae: 0.2428 - root_mean_squared_error: 0.5984 - mean_squared_logarithmic_error: 0.0772
Epoch 7/100
23/23 [==============================] - 0s 7ms/step - loss: 0.3536 - mse: 0.3536 - mae: 0.2461 - root_mean_squared_error: 0.5947 - mean_squared_logarithmic_error: 0.0806
Epoch 8/100
23/23 [==============================] - 0s 6ms/step - loss: 0.3429 - mse: 0.3429 - m

23/23 [==============================] - 0s 7ms/step - loss: 0.2594 - mse: 0.2594 - mae: 0.1748 - root_mean_squared_error: 0.5093 - mean_squared_logarithmic_error: 0.0614
Epoch 48/100
23/23 [==============================] - 0s 6ms/step - loss: 0.2488 - mse: 0.2488 - mae: 0.1778 - root_mean_squared_error: 0.4988 - mean_squared_logarithmic_error: 0.0597
Epoch 49/100
23/23 [==============================] - 0s 7ms/step - loss: 0.2820 - mse: 0.2820 - mae: 0.1753 - root_mean_squared_error: 0.5311 - mean_squared_logarithmic_error: 0.0677
Epoch 50/100
23/23 [==============================] - 0s 6ms/step - loss: 0.2531 - mse: 0.2531 - mae: 0.1753 - root_mean_squared_error: 0.5031 - mean_squared_logarithmic_error: 0.0605
Epoch 51/100
23/23 [==============================] - 0s 6ms/step - loss: 0.2478 - mse: 0.2478 - mae: 0.1779 - root_mean_squared_error: 0.4978 - mean_squared_logarithmic_error: 0.0593
Epoch 52/100
23/23 [==============================] - 0s 7ms/step - loss: 0.2351 - mse: 0.235

23/23 [==============================] - 0s 7ms/step - loss: 0.2664 - mse: 0.2664 - mae: 0.1913 - root_mean_squared_error: 0.5161 - mean_squared_logarithmic_error: 0.0639
Epoch 92/100
23/23 [==============================] - 0s 6ms/step - loss: 0.2355 - mse: 0.2355 - mae: 0.1866 - root_mean_squared_error: 0.4853 - mean_squared_logarithmic_error: 0.0574
Epoch 93/100
23/23 [==============================] - 0s 6ms/step - loss: 0.2217 - mse: 0.2217 - mae: 0.1892 - root_mean_squared_error: 0.4708 - mean_squared_logarithmic_error: 0.0570
Epoch 94/100
23/23 [==============================] - 0s 6ms/step - loss: 0.2343 - mse: 0.2343 - mae: 0.1779 - root_mean_squared_error: 0.4840 - mean_squared_logarithmic_error: 0.0551
Epoch 95/100
23/23 [==============================] - 0s 6ms/step - loss: 0.2328 - mse: 0.2328 - mae: 0.1881 - root_mean_squared_error: 0.4825 - mean_squared_logarithmic_error: 0.0569
Epoch 96/100
23/23 [==============================] - 0s 7ms/step - loss: 0.2283 - mse: 0.228

In [22]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/propv185/'

In [23]:
total_training_time

[62.87585949897766,
 62.31042122840881,
 61.25686550140381,
 21.014124393463135,
 21.75888228416443,
 22.087533473968506,
 22.210252285003662,
 22.36575698852539,
 21.549898862838745,
 22.045281887054443]

In [24]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 11.315829213460287


In [25]:
total_training_loss_history

[{'loss': [0.48507317900657654,
   0.4519845247268677,
   0.43430987000465393,
   0.41192182898521423,
   0.37723854184150696,
   0.35991957783699036,
   0.3525245785713196,
   0.3420819044113159,
   0.31336700916290283,
   0.32836249470710754,
   0.32049235701560974,
   0.31330716609954834,
   0.32575082778930664,
   0.32314333319664,
   0.31182628870010376,
   0.30643168091773987,
   0.2888124883174896,
   0.2968587875366211,
   0.2829658091068268,
   0.2868146300315857,
   0.2949467897415161,
   0.2790922224521637,
   0.2844255864620209,
   0.2776477336883545,
   0.28438296914100647,
   0.2851651608943939,
   0.2751768231391907,
   0.2857990562915802,
   0.2735907733440399,
   0.2736872136592865,
   0.271337628364563,
   0.28313809633255005,
   0.2693462669849396,
   0.2605923116207123,
   0.2636527419090271,
   0.26557594537734985,
   0.2566834092140198,
   0.24444593489170074,
   0.2668896019458771,
   0.2764933705329895,
   0.2815673351287842,
   0.2875663936138153,
   0.29932487

In [26]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(10):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 0.08349429965019226
avg mae of training: 0.06105696062246958


In [27]:
total_scores

[[0.32140326499938965,
  0.32140326499938965,
  0.19581151008605957,
  0.5669243931770325,
  0.07376454770565033],
 [0.28570252656936646,
  0.28570252656936646,
  0.19628216326236725,
  0.5345115065574646,
  0.07141554355621338],
 [0.30492132902145386,
  0.30492132902145386,
  0.198886901140213,
  0.5521968007087708,
  0.07086735218763351],
 [0.3496015667915344,
  0.3496015667915344,
  0.21444451808929443,
  0.591271162033081,
  0.07964641600847244],
 [0.3319781422615051,
  0.3319781422615051,
  0.23236872255802155,
  0.5761754512786865,
  0.08281941711902618],
 [0.2858119606971741,
  0.2858119606971741,
  0.1915985345840454,
  0.534613847732544,
  0.06907180696725845],
 [0.26097825169563293,
  0.26097825169563293,
  0.19239310920238495,
  0.5108603239059448,
  0.0663246288895607],
 [0.34476569294929504,
  0.34476569294929504,
  0.22832846641540527,
  0.587167501449585,
  0.08758220076560974],
 [0.3256637156009674,
  0.3256637156009674,
  0.14513275027275085,
  0.5706695318222046,
  0.

In [28]:
a = [[1.1072313785552979,
  1.1072313785552979,
  0.45697730779647827,
  1.052250623703003,
  0.1505131721496582],
 [1.0949896574020386,
  1.0949896574020386,
  0.4038865566253662,
  1.0464175939559937,
  0.12835438549518585],
 [1.1006139516830444,
  1.1006139516830444,
  0.4063127636909485,
  1.04910147190094,
  0.13611803948879242],
 [1.0576918125152588,
  1.0576918125152588,
  0.40475377440452576,
  1.0284414291381836,
  0.132333904504776],
 [1.087944507598877,
  1.087944507598877,
  0.45055320858955383,
  1.0430457592010498,
  0.14320626854896545],
 [1.1228747367858887,
  1.1228747367858887,
  0.4579310417175293,
  1.0596578121185303,
  0.15697523951530457],
 [1.1081269979476929,
  1.1081269979476929,
  0.41065943241119385,
  1.0526760816574097,
  0.13614483177661896],
 [1.0938034057617188,
  1.0938034057617188,
  0.38377878069877625,
  1.0458505153656006,
  0.12563826143741608],
 [1.046379804611206,
  1.046379804611206,
  0.3927305042743683,
  1.0229270458221436,
  0.12750454246997833],
 [1.077500343322754,
  1.077500343322754,
  0.4189930856227875,
  1.0380271673202515,
  0.1433822512626648],
 [1.0831170082092285,
  1.0831170082092285,
  0.39570093154907227,
  1.04072904586792,
  0.12718339264392853],
 [1.0793119668960571,
  1.0793119668960571,
  0.4216558337211609,
  1.0388994216918945,
  0.13091830909252167],
 [1.0471224784851074,
  1.0471224784851074,
  0.41589176654815674,
  1.0232900381088257,
  0.1298009604215622],
 [1.112039566040039,
  1.112039566040039,
  0.432132750749588,
  1.0545328855514526,
  0.14624285697937012],
 [1.117440104484558,
  1.117440104484558,
  0.3917667865753174,
  1.057090401649475,
  0.12741179764270782],
 [1.0524473190307617,
  1.0524473190307617,
  0.4215909242630005,
  1.0258885622024536,
  0.1336907595396042],
 [1.0733959674835205,
  1.0733959674835205,
  0.42276591062545776,
  1.0360482931137085,
  0.13468603789806366],
 [1.1121301651000977,
  1.1121301651000977,
  0.4868672788143158,
  1.054575800895691,
  0.16745002567768097],
 [1.0329020023345947,
  1.0329020023345947,
  0.3979037404060364,
  1.0163178443908691,
  0.12879155576229095],
 [1.0707591772079468,
  1.0707591772079468,
  0.4306138753890991,
  1.034774899482727,
  0.13812918961048126],
 [1.075225830078125,
  1.075225830078125,
  0.40528926253318787,
  1.0369309186935425,
  0.12670773267745972],
 [1.083695411682129,
  1.083695411682129,
  0.4407236576080322,
  1.0410069227218628,
  0.13586269319057465],
 [1.0840530395507812,
  1.0840530395507812,
  0.4021536111831665,
  1.0411787033081055,
  0.12666361033916473],
 [1.1058012247085571,
  1.1058012247085571,
  0.3986241817474365,
  1.0515708923339844,
  0.12572358548641205],
 [1.1244145631790161,
  1.1244145631790161,
  0.4341464340686798,
  1.0603841543197632,
  0.14787453413009644],
 [1.0968248844146729,
  1.0968248844146729,
  0.41786739230155945,
  1.047294020652771,
  0.13397936522960663],
 [1.0843594074249268,
  1.0843594074249268,
  0.4130922853946686,
  1.0413258075714111,
  0.13689492642879486],
 [1.0843403339385986,
  1.0843403339385986,
  0.4019320011138916,
  1.0413166284561157,
  0.13194996118545532],
 [1.0823019742965698,
  1.0823019742965698,
  0.3958171606063843,
  1.0403374433517456,
  0.13313862681388855],
 [1.056094765663147,
  1.056094765663147,
  0.4249243140220642,
  1.0276646614074707,
  0.13509279489517212]]

In [29]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])

In [30]:
test_mae

[0.45697730779647827,
 0.4038865566253662,
 0.4063127636909485,
 0.40475377440452576,
 0.45055320858955383,
 0.4579310417175293,
 0.41065943241119385,
 0.38377878069877625,
 0.3927305042743683,
 0.4189930856227875]

In [31]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(10):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 0.10460398594538371
avg mae: 0.06643135100603104
avg are: 0.027753195663293203


In [32]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.2219405323266983
median testing mse: 0.32353349030017853


In [33]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.17954128980636597
median testing mae: 0.19698800891637802


In [34]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.0831858366727829


In [35]:
test_ARE

[0.07669617,
 0.07581121,
 0.0859882,
 0.08333333,
 0.07920354,
 0.08303835,
 0.09129793,
 0.07920354,
 0.0920354,
 0.0859882]